In [19]:
import pandas as pd
import numpy as np
import re

#reading the txt in a list
with open('test.txt') as f:
    lines = f.readlines()

#Creating a function for removing unknown number of dots
def remove_dots(lines):
    return re.sub('\.+', '.', lines)

def reset_idx(info):
    return info.reset_index().drop("index", axis = 1)

def get_time(finish, start):
    """
    This function returns the total minutes between two  points in time interval
    """
    diff = finish - start
    days, seconds = diff.days, diff.seconds
    total_minutes = days * 1440 + seconds // 60
    return total_minutes

    

for i in range(len(lines)):
    #Removing the "/n" from each line
    lines[i] = lines[i][:-1]
    #Splitting the data 
    lines[i] = lines[i].split(sep="  ")
    #Removing unnecessary charcters
    lines[i][0] = lines[i][0][:-1]
    #Getting the second date_time value
    index_date = lines[i][1].find(lines[i][0][0:2])
    lines[i].append(lines[i][1][index_date:index_date+19])
    #Deleting the unnecessary data
    lines[i][1] = lines[i][1].replace(lines[i][2], "")
    #Removing the dots from the string
    lines[i][1] = remove_dots(lines[i][1])
    #Appending the type of enterance
    lines[i].append(lines[i][1][lines[i][1].find(".")+1:])
    #Removing the dot and the entrance type
    lines[i][1] = lines[i][1].replace(lines[i][1][lines[i][1].find("."):], '')

#Converting the created list to a dataframe for easy handling
df = pd.DataFrame(lines, columns = ["System_date_time", "Name", "Actual_date_time", "type"])

#Removing the data with no importance and reseting the indexes
df = df.loc[(df["Name"] != "715[192") & (df["Name"] != "704[192")]
df = df.reset_index().drop("index", axis = 1)

from datetime import datetime

def transform_date(date_str): 
    '''
    This function transforms the string of the system date and time to a datetime object
    '''
    return datetime.strptime(date_str, '%d/%m/%Y %H:%M:%S')

def transform_date2(date_str): 
    '''
    This function transforms the string of the actual date and time to a datetime object
    '''
    return datetime.strptime(date_str, '%d.%m.%Y %H:%M:%S')


#Transforming the system and actual date_time strings to datetime objects
df["System_date_time"] = df["System_date_time"].apply(transform_date)
df["Actual_date_time"] = df["Actual_date_time"].apply(transform_date2)

#Creating two new columns for just the date and just the time of the entrance
df["date"] = df["Actual_date_time"][0].date()
df["time"] = df["Actual_date_time"][0].time()
#Assigning the date and time for each case
for i in range(len(df)):
    df["date"][i] = df["Actual_date_time"][i].date()
    df["time"][i] = df["Actual_date_time"][i].time()

df

C:\Users\balam\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\balam\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,System_date_time,Name,Actual_date_time,type,date,time
0,2021-10-25 06:43:18,Dwight Schrute,2021-10-25 06:43:11,CheckIn,2021-10-25,06:43:11
1,2021-10-25 07:00:25,Michael Scott,2021-10-25 07:00:25,CheckIn,2021-10-25,07:00:25
2,2021-10-25 07:22:29,Jim Halpert,2021-10-25 07:22:23,CheckIn,2021-10-25,07:22:23
3,2021-10-25 07:25:34,Pam Beesly,2021-10-25 07:25:34,CheckIn,2021-10-25,07:25:34
4,2021-10-25 07:25:40,Pam Beesly,2021-10-25 07:25:35,CheckIn,2021-10-25,07:25:35
...,...,...,...,...,...,...
131,2021-10-28 12:36:59,Angela Martin,2021-10-28 12:36:53,CheckOut,2021-10-28,12:36:53
132,2021-10-28 12:46:59,Michael Scott,2021-10-28 12:47:00,CheckIn,2021-10-28,12:47:00
133,2021-10-28 12:49:00,Dwight Schrute,2021-10-28 12:48:54,CheckOut,2021-10-28,12:48:54
134,2021-10-28 13:21:28,Dwight Schrute,2021-10-28 13:21:22,CheckIn,2021-10-28,13:21:22


In [20]:
data = pd.DataFrame(columns = ["day", "name", "worked_hours", "first_checkin", "final_checkout",
                       "break_time", "break_start", "break_finish"])
data

,day,name,worked_hours,first_checkin,final_checkout,break_time,break_start,break_finish


In [26]:
"""
So after analyzing the data I indentified some edge cases like multiple Checkins at the same time, or Checkouts
Also in some cases the Checkout was misunderstood with a checkin so I tried removing those cases

"""
#First iterate through each unique date
for elem in df["date"].unique():
    test = df.loc[df["date"] == elem]
    #Afterwards iterate through each member that entered
    for pers in test["Name"].unique():
        #Creating a dataframe for the iteration member for removing edgecases
        info = test.loc[test["Name"] == pers]
        print(info)
        #Resetting the indexes fo easy iteration
        info = reset_idx(info)
        #First check if there is any checkout or checkin, if the day did not end in many cases there is the possibility of not finishing the day
        if "CheckOut" in info["type"].value_counts():
            nr_checkout = info["type"].value_counts()["CheckOut"]
        if "CheckIn" in info["type"].value_counts():
            nr_checkin = info["type"].value_counts()["CheckIn"]
        #Checking if the number of checkin is equal to the checkouts to verify if there are problems 
        if nr_checkout != nr_checkin:
            for i in range(len(info)-1):
                #Checking if there are repetitive checkins or checkouts in the same interval of time and removing them 
                if (info["type"][i] == info["type"][i+1]) and (get_time(info["Actual_date_time"][i+1], info["Actual_date_time"][i])<10):
                    info = info.drop(i, axis = 0)
            info = reset_idx(info)
            for i in range(len(info)-2):
                #Checking if there are three interferencees of the same type with a big distance between them and replacing the middle one with opposite type
                if (info["type"][i] == info["type"][i+1]) and (info["type"][i] == info["type"][i+2]) and (get_time(info["Actual_date_time"][i+1], info["Actual_date_time"][i])>30):
                    if info["type"][i+1]=="CheckIn":
                        info["type"][i+1]="CheckOut"
                    else:
                        info["type"][i+1]="CheckIn"

        

      System_date_time            Name    Actual_date_time      type  \
0  2021-10-25 06:43:18  Dwight Schrute 2021-10-25 06:43:11   CheckIn   
23 2021-10-25 15:20:02  Dwight Schrute 2021-10-25 15:19:56  CheckOut   

          date      time  
0   2021-10-25  06:43:11  
23  2021-10-25  15:19:56  
      System_date_time           Name    Actual_date_time      type  \
1  2021-10-25 07:00:25  Michael Scott 2021-10-25 07:00:25   CheckIn   
21 2021-10-25 15:14:23  Michael Scott 2021-10-25 15:14:24  CheckOut   
24 2021-10-25 15:59:09  Michael Scott 2021-10-25 15:59:10   CheckIn   
35 2021-10-25 18:24:28  Michael Scott 2021-10-25 18:24:28  CheckOut   

          date      time  
1   2021-10-25  07:00:25  
21  2021-10-25  15:14:24  
24  2021-10-25  15:59:10  
35  2021-10-25  18:24:28  
      System_date_time         Name    Actual_date_time      type        date  \
2  2021-10-25 07:22:29  Jim Halpert 2021-10-25 07:22:23   CheckIn  2021-10-25   
25 2021-10-25 16:06:20  Jim Halpert 2021-10-25 16

C:\Users\balam\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



       System_date_time             Name    Actual_date_time      type  \
94  2021-10-27 11:24:33  Toby Flenderson 2021-10-27 11:24:28   CheckIn   
110 2021-10-27 19:31:27  Toby Flenderson 2021-10-27 19:31:22  CheckOut   

           date      time  
94   2021-10-27  11:24:28  
110  2021-10-27  19:31:22  
       System_date_time            Name    Actual_date_time      type  \
112 2021-10-28 06:57:15  Dwight Schrute 2021-10-28 06:57:10   CheckIn   
133 2021-10-28 12:49:00  Dwight Schrute 2021-10-28 12:48:54  CheckOut   
134 2021-10-28 13:21:28  Dwight Schrute 2021-10-28 13:21:22   CheckIn   

           date      time  
112  2021-10-28  06:57:10  
133  2021-10-28  12:48:54  
134  2021-10-28  13:21:22  
       System_date_time           Name    Actual_date_time      type  \
113 2021-10-28 07:03:28  Michael Scott 2021-10-28 07:03:30   CheckIn   
130 2021-10-28 12:34:08  Michael Scott 2021-10-28 12:34:10  CheckOut   
132 2021-10-28 12:46:59  Michael Scott 2021-10-28 12:47:00   CheckIn   

In [22]:

def get_info(temp):
    """
    This is the main analyzing function for the dataframe
    """
    #First we assign the date
    date = temp["date"][0]
    #Afterwards get the first checkin and last checkout 
    first_checkin = temp["time"][0]
    final_checkout = temp["time"][len(temp)-1]
    
    total_minutes = 0
    brk_start = 0
    brk_end = 0
    brk_time = 0
    #Veryfing if there was any break
    if len(temp)>2:
        #Assigning the start and the end of the break
        brk_start = temp["time"][1]
        brk_end = temp["time"][len(temp)-2]
        #Calculating the time of the break
        brk_time = temp["Actual_date_time"][len(temp)-2] - temp["Actual_date_time"][1]
        if brk_time!=0:
            brk_time = brk_time.days*1440 + brk_time.seconds // 60
        
    #Iterating through the array to get the time from each interval
    for i in range(len(temp)-1, 0, -2):
        diff = temp["Actual_date_time"][i] - temp["Actual_date_time"][i-1]
        total_minutes += get_time(temp["Actual_date_time"][i], temp["Actual_date_time"][i-1])
    
    return [date, temp["Name"][0], total_minutes//60, first_checkin, final_checkout, brk_time, brk_start, brk_end]


#Appending the data to the new dataframe
for elem in df["date"].unique():
    test = df.loc[df["date"] == elem]
    for pers in test["Name"].unique():
        to_append = get_info(reset_idx(test.loc[test["Name"] == pers]))
        a_series = pd.Series(to_append, index = data.columns)
        data = data.append(a_series, ignore_index=True)
        
data = reset_idx(data)
data

,day,name,worked_hours,first_checkin,final_checkout,break_time,break_start,break_finish
0,2021-10-25,Dwight Schrute,8,06:43:11,15:19:56,0,0,0
1,2021-10-25,Michael Scott,10,07:00:25,18:24:28,44,15:14:24,15:59:10
2,2021-10-25,Jim Halpert,8,07:22:23,16:06:14,0,0,0
3,2021-10-25,Pam Beesly,8,07:25:34,16:23:46,300,07:25:35,12:26:34
4,2021-10-25,Ryan Howard,8,07:52:20,16:51:23,0,0,0
5,2021-10-25,Andy Bernard,8,08:12:59,16:22:39,0,0,0
6,2021-10-25,Angela Martin,8,08:16:44,16:51:29,0,0,0
7,2021-10-25,Kelly Kapoor,8,08:29:54,17:05:38,0,0,0
8,2021-10-25,Toby Flenderson,10,08:29:48,18:30:38,0,0,0
9,2021-10-25,Creed Bratton,8,08:52:16,17:34:34,0,0,0


In [23]:
#Exporting the json file
data.to_json('export.json', orient = 'index')